# research_code

> A selection of useful code chunks for analysing THz-TDS data.

In [ ]:
#| hide
from fastcore.test import *
import fastcore.docments as docment
from nbdev.showdoc import DocmentTbl

In [ ]:
#| default_exp research_code

In [ ]:
#| export 
import numpy as np
from scipy import constants as consts
from scipy.optimize import curve_fit
from scipy.stats import chisquare
from scipy.fft import fft,fftfreq,rfft, fftshift,fftn
from scipy.interpolate import interp1d
import inspect
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob,os,re,pathlib
import seaborn as sns
from scipy.signal import savgol_filter
from alive_progress import alive_it, alive_bar
import time

In [ ]:
#| export
def exp_decay(x: np.ndarray, # Input array that contains the indpendent variable.
    Amplitude: np.single, # Pre-factor of the exponential decay. 
    tau: np.single, # Decay constant.
    x0: np.single, # Offset for the independent variable .
    ) -> np.ndarray: # A new float array containinsg the exponentially decaying function.
    '''
    Single exponentially decaying function.
    '''
    decayOutput = Amplitude * np.exp(-(x-x0)/tau)
    
    return decayOutput

In [ ]:
#| export
def biexp_decay_beta(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude1: np.single, # Prefactor of the first exponential decay.
    tau1: np.single, # Decay constant of the first exponential term.
    beta: np.single, # Ratio that decays via channel1 or channel2
    tau2: np.single, # Decay constant of the second exponential term.
    x0: np.single, # Offset for the indpendent variable.
    ) -> np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Bi-exponentially decaying function.
    '''
    decayOutput = Amplitude1 * (beta * np.exp(-(x-x0)/tau1) + (1 - beta) * np.exp(-(x-x0)/tau2))
    
    return decayOutput

In [ ]:
#| export
def biexp_decay(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude1: np.single, # Prefactor of the first exponential decay.
    tau1: np.single, # Decay constant of the first exponential term.
    Amplitude2: np.single, # Prefactor of the second exponential decay.
    tau2: np.single, # Decay constant of the second exponential term.
    x0: np.single, # Offset for the indpendent variable.
    ) -> np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Bi-exponentially decaying function.
    '''
    decayOutput = Amplitude1 * np.exp(-(x-x0)/tau1) + Amplitude2 * np.exp(-(x-x0)/tau2)
    
    return decayOutput

In [ ]:
#| export
def gaussian(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude: np.single, # Prefactor of the gaussian.
    sigma: np.single, # Standard deviation of the gaussian.
    x0: np.single, # Offset for the indpendent variable.
    ) -> np.ndarray: # A new float array containing the gaussian function.
    '''
    Gaussian function.
    '''
    outputGaussian = Amplitude * np.exp(-((x-x0)/sigma)**2)

    return outputGaussian

In [ ]:
#| export
def convolved_exp_decay(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude: np.single, # Prefactor of the exponential decay.
    tau: np.single, # Decay constant.
    sigma: np.single, # Standard deviation of the gaussian
    x0: np.single, # Offset for the indpendent variable.
    y0: np.single, # Offset in y.
    ) -> np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Single exponentially decaying function convolved with a gaussian.
    '''
    L = len(x) # length of the independent variable.

    exp_term = exp_decay(x,1,tau,x0) # The exponential decay.

    gaussian_term = gaussian(x,1.0,sigma,x0) # The gaussian part, setting the amplitude normalised to 1.0.
    
    convolved = np.convolve(exp_term,gaussian_term,mode='full')[0:L] # convolved function.

    convolved = Amplitude*convolved/np.max(convolved) # Normalise the function so the maximum is at Amplitude
    
    return convolved + y0

In [ ]:
#| export
def convolved_biexp_decay(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude1:np.single, # Prefactor of the first exponential decay.
    tau1:np.single, # Decay constant of the first exponential term.
    Amplitude2:np.single, # Prefactor of the second exponential decay.
    tau2:np.single, # Decay constant of the second exponential term.
    sigma:np.single, # Standard deviation of the gaussian
    x0:np.single, # Offset for the indpendent variable.
    y0:np.single, # Offset in y.
    ) ->np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Bi-exponentially decaying function convolved with a gaussian.
    '''    
    L = len(x) # length of the independent variable.

    exp_term1 = exp_decay(x,1,tau1,x0) # The biexponential decay.

    exp_term2 = exp_decay(x,1,tau2,x0) # The biexponential decay.

    gaussian_term = gaussian(x,1.0,sigma,x0) # The gaussian part, setting the amplitude normalised to 1.0.
    
    convolved1 = np.convolve(gaussian_term,exp_term1,mode='full')[0:L]
    convolved2 = np.convolve(gaussian_term,exp_term2,mode='full')[0:L]# convolved function.

    convolved = Amplitude1*convolved1/np.max(convolved1) + Amplitude2*convolved2/np.max(convolved2)

    #convolved *= (Amplitude1+Amplitude2-y0)/np.max(convolved) # Normalise the function so the maximum is at Amplitude
    
    return convolved + y0

In [ ]:
#| export
def convolved_biexp_decay_beta(x:np.ndarray, # Input array that contains the indpendent variable.
    Amplitude1:np.single, # Prefactor of the first exponential decay.
    tau1:np.single, # Decay constant of the first exponential term.
    beta:np.single, # Prefactor of the second exponential decay.
    tau2:np.single, # Decay constant of the second exponential term.
    sigma:np.single, # Standard deviation of the gaussian
    x0:np.single, # Offset for the indpendent variable.
    y0:np.single, # Offset in y.
    ) ->np.ndarray: # A new float array containing the exponentially decaying function.
    '''
    Bi-exponentially decaying function, with a beta term, convolved with a gaussian.
    '''    
    L = len(x) # length of the independent variable.

    gaussian_term = gaussian(x,1.0,sigma,x0) # The gaussian part, setting the amplitude normalised to 1.0.
    
    exp_term1 = np.exp(-(x-x0)/tau1)

    exp_term2 = np.exp(-(x-x0)/tau2)

    convolved_term1 = np.convolve(gaussian_term,exp_term1,mode='full')[0:L]

    convolved_term2 = np.convolve(gaussian_term,exp_term2,mode='full')[0:L]
    
    convolved = beta * convolved_term1/np.max(convolved_term1)

    convolved += (1 - beta) * convolved_term2/np.max(convolved_term2)
    
    return convolved + y0

In [ ]:
#| export
def rebin(x,y,m):
    x = np.array(x)
    y = np.array(y)
    xnew = np.linspace(x[0],x[-1],int(round(len(x)/m)*m))
    yinterp = np.interp(xnew, x, y)
    
    window_sz = m
    ym = yinterp[:int(len(yinterp)/window_sz*window_sz)].reshape(-1,window_sz).mean(1)
    xm = np.linspace(xnew[0],xnew[-1],int(len(xnew)/m))

    return (xm,ym)

In [ ]:
#| export
def R2(xdata,ydata,residual):
        ss_tot = np.sum((ydata-np.mean(ydata))**2)
        ss_res = np.sum(residual**2)
        return (1 - ss_res/ss_tot)

In [ ]:
#| export
def R2_adj(xdata,ydata,residual,popt):
        N_points = len(xdata)
        nos_fit_params = len(popt)
        ss_tot = np.sum((ydata-np.mean(ydata))**2)
        ss_res = np.sum(residual**2)
        R2 = 1 - ss_res/ss_tot
        R2_adj = 1 - (1 - R2)*(N_points-1)/(N_points-nos_fit_params-1)
        return R2_adj

In [ ]:
#| export
def X2_adj(xdata,ydata,residual,named_function,popt):
    N_points = len(xdata)
    nos_fit_params = len(popt)
    rss = np.sum(residual**2)
    x2_adj = rss / (N_points-nos_fit_params)
    return x2_adj

In [ ]:
#| export
def X2(xdata,ydata,residual,named_function,popt):
    expected_values = named_function(xdata,*popt)
    return np.sum((1/expected_values) * residual**2)

In [ ]:
#| export
def R_standardised(xdata,ydata,residual,named_function,popt):

    N_points = len(xdata)
    
    nos_fit_params = len(popt)
    
    RSS = np.sum(residual**2)

    RSE = np.sqrt(RSS/(N_points - nos_fit_params - 1))

    X = np.vstack([residual,np.ones(len(residual))]).T

    h_leverage = np.diag(X @ np.linalg.inv(X.T @ X) @ X.T)

    r_standardised = residual/np.sqrt(RSE*np.sqrt(1-h_leverage))

    return r_standardised

In [ ]:
#| export
class fit_result:
    def __init__(self,xdata,ydata,named_function,popt,pcov):
        self.popt = popt
        self.pstd = np.sqrt(np.diag(pcov))
        self.residual = ydata-named_function(xdata,*popt)
        self.R2 = R2(xdata,ydata,self.residual)
        self.R2_adj = R2_adj(xdata,ydata,self.residual,popt)
        self.RMSE = np.sqrt(np.sum(self.residual**2) / (len(xdata) - len(popt)))
        self.chi2_adj = X2_adj(xdata,ydata,self.residual,named_function,popt)
        self.r_standardised = R_standardised(xdata,ydata,self.residual,named_function,popt)

In [ ]:
#| export
def interp_data(x:np.ndarray, # independent variable
                y:np.ndarray, # dependent variable
               ): # returns interpolated data for minimum delta
    interp_f = interp1d(x,y)
    xx_new = np.arange(np.min(x),np.max(x),np.diff(x).min()*1)
    yy_new = interp_f(xx_new)

    return (xx_new,yy_new)

In [ ]:
#| export
def fit_curve(named_function, # the fit function
              x:np.ndarray, # independent variable 
              y:np.ndarray, # dependent variable
              **kwargs) -> object: # returns a an object of the class 'fit_object'

    (xnew,ynew) = interp_data(x,y)
    
    if 'guess' in list(kwargs.keys()):
        guess = kwargs['guess']
    else:
        return None

    if 'method' in list(kwargs.keys()):
        method = kwargs['method']
    else:
        method = 'lm'
        
    def fmod(named_function,x,*args,**kwargs):

        all_args = inspect.getfullargspec(named_function).args
        sub_args = list(kwargs.keys()) # just the names of the fixed variables
        sub_vals = list(kwargs.values())
    
        all_args.pop(0)
        new_kwargs = {}
        truth = tuple(val in sub_args for val in all_args)
        #truth = np.logical_not(truth)
        j=0
        largs = list(args)
        
        for i,arg in enumerate(all_args):
            if truth[i]==True:
                new_kwargs[arg] = kwargs[arg]
            elif truth[i]==False:
                new_kwargs[arg] = largs[j]
                # print(largs[j])
                j+=1
                
        return named_function(x,**new_kwargs)
    
    all_args = np.array(named_function.__code__.co_varnames)

    ([*keys],vals) = zip(*guess.items())
    
    guess2,truth = np.ndarray.flatten(np.array(vals).T).reshape(2,len(keys))

    # print(guess2,truth)

    truth = truth.astype(int)
    not_truth = np.logical_not(truth).astype(int)

    key_mask = np.ma.masked_array(keys,mask=not_truth).compressed()
    guess_mask = np.ma.masked_array(guess2,mask=not_truth).compressed() # the fixed values
    guess_mask_float = np.ma.masked_array(guess2,mask=truth).compressed() # the non-fixed values

    kwargs = dict(zip(key_mask,guess_mask))

    fun2 = lambda x,*args: fmod(named_function,x,*args,**kwargs) 
    
    popt,pcov = curve_fit(fun2,xnew,ynew,p0=guess_mask_float,method=method)

    fit_object = fit_result(x,y,fun2,popt,pcov)
    
    pstd = np.sqrt(np.diag(pcov))

    popt_new = np.zeros(len(guess2))
    pstd_new = np.zeros(len(guess2))

    j=0
    k=0
    for i,val in enumerate(truth):
        if val==False:
            popt_new[i] = popt[j]
            pstd_new[i] = pstd[j]
            j+=1
        else:
            popt_new[i] = guess_mask[k]
            pstd_new[i] = 0.0
            k+=1

    fit_object.popt = popt_new
    fit_object.pstd = pstd_new
    return fit_object

In [ ]:
#| export
def fit_trace2(x:np.ndarray, # the independent variable (e.g. x,time), usually time here
              y:np.ndarray, # the dependent variable (e.g. y)
              trange:tuple, # the x range to fit the data over
              guess:dict, # a dictionary containing the guess parameters for the fit
              named_function, # the fit function
             ) -> tuple:

    tol = np.abs(x[1]-x[0])
    t_start = np.where(np.abs(x-trange[0])<tol)[0][0]
    t_end = np.where(np.abs(x-trange[1])<tol)[0][0]
    
    x_sub = x[t_start:t_end]
    y_sub = y[t_start:t_end]
    
    popt,pcov = curve_fit(named_function,x_sub,y_sub,p0=guess,method='lm')

    fit_object = fit_result(x_sub,y_sub,named_function,popt,pcov)
    
    return fit_object

In [ ]:
#| export
def global_fit(named_function, # the name of the function being fitted
               x:dict, # Input array that contains the indpendent variable
               y:dict, # dependent variable, can be a matrix
               guess:dict, # guess parameters in a dictionary, with gobal as True or false
              ) ->np.ndarray: # A new float array containing the exponentially decaying function.

    '''
    The purpose of this function is to allow for the simple global fitting of data
    '''
    
    ([*keys],vals) = zip(*guess.items())
    
    guess2,truth_global = np.ndarray.flatten(np.array(vals).T).reshape(2,len(keys))
    
    N = len(guess2) # the full number of guess values

    ([*keys_x],[*vals_x])=zip(*x.items())
    ([*keys_y],[*vals_y])=zip(*y.items())

    M = len(keys_x)
    L = [len(vals_x[i]) for i in range(len(keys_x))]

    # print(f'This had better be OK: {L}')
    
    
    # M,L = np.shape(y) # will give the number of points in each plot (L) and the number of different traces (M). This assumes that there is the same
    # number of points in each run...

    for i in range(M):
        if i==0:
            new_x = np.array(vals_x[i])
            new_y = np.array(vals_y[i])
        else:
            new_x = np.concatenate([new_x,vals_x[i]])
            new_y = np.concatenate([new_y,vals_y[i]])
    #     new_x = np.ndarray.flatten(np.array(vals_x)) # will give an x vector of same length as new_y
    # new_y = np.ndarray.flatten(np.array(vals_y)) # flatten all the data into one long column vector

    all_args = inspect.getfullargspec(named_function).args

    N_args = len(all_args) - 1

    truth_global = truth_global.astype(int) #

    
    for j in range(N_args):
        if truth_global[j]==True:
            truth_global[j] = False
            for k in range(1,M):
                truth_global[j+k*N_args]=True
        else:
            pass
        
    truth_local = np.logical_not(truth_global).astype(int)

    key_mask_local = np.ma.masked_array(keys,mask=truth_global).compressed()
    guess_mask_local = np.ma.masked_array(guess2,mask=truth_global).compressed() # the unfixed values
    guess_mask_global = np.ma.masked_array(guess2,mask=truth_local).compressed() # the fixed values

    first_vals = guess2[:N_args]

    def builder(x,*args):
        
        ypiece = np.zeros(len(x))

        #print(f'M: {M}')
        args2 = np.zeros(N_args)
        Nargs = len(args)
        v=0
        #print(L)
        for i in range(M):

            for j,val in enumerate(truth_local[i*N_args:(i+1)*N_args]):

                if val==0:
                    args2[j] = args[j]
                    
                else:
                    args2[j] = args[v]
                    v+=1
            if i==0:
                start_count = 0
                end_count = L[i]
            else:
                start_count = int(np.sum(L[0:i])+1)
                end_count = int(np.sum(L[0:i+1]))
                # print(start_count,end_count)
            
            ypiece[start_count:end_count] = named_function(x[start_count:end_count],*args2)
            #print(f'ya: {ypiece}')
        return ypiece

    if named_function==convolved_biexp_decay_beta:
        L_beta = N_args
        M_beta = len(truth_global)
        
        b_low = -np.inf * np.ones(M_beta)
        b_high = np.inf * np.ones(M_beta)
        
        for j in range(2,M_beta,L_beta):
            b_low[j] = 0
            b_high[j] = 1

        for j in range(1,M_beta,L_beta):
            b_low[j] = 0

        for j in range(3,M_beta,L_beta):
            b_low[j] = 0
        
        bounds_lower = np.ma.masked_array(b_low,mask=truth_global).compressed()
        bounds_upper = np.ma.masked_array(b_high,mask=truth_global).compressed()

        print(guess_mask_local,bounds_lower,bounds_upper)
        popt,pcov = curve_fit(builder,new_x,new_y,p0=guess_mask_local,bounds=(bounds_lower,bounds_upper),method='trf')
        
    else:
        popt,pcov = curve_fit(builder,new_x,new_y,p0=guess_mask_local,method='lm')

    fit_object = fit_result(new_x,new_y,builder,popt,pcov)

    pstd = np.sqrt(np.diag(pcov))

    popt_new = np.zeros(len(guess2))
    pstd_new = np.zeros(len(guess2))

    j=0
    for i,val in enumerate(truth_global):
        if val==False:
            popt_new[i] = popt[j]
            pstd_new[i] = pstd[j]
            j+=1
        else:
            popt_new[i] = popt[i-(i//N_args)*N_args]
            pstd_new[i] = 0.0

    popt_new = np.reshape(popt_new,(M,N_args))

    pstd_new = np.reshape(pstd_new,(M,N_args))

    fit_object.popt = popt_new

    fit_object.pstd = pstd_new

    trace_new = ['trace'+str(i) for i in range(M)]

    dict_test = {'trace'+str(i):[(round(popt_new[i,j],4),round(pstd_new[i,j],4)) for j in range(N_args)] for i in range(M)}

    df = pd.DataFrame(dict_test)
    df['name'] = all_args[1:]
    # test = dict(zip(keys,popt_new))
    df.set_index('name',inplace=True)
    df.index.name = None
    df['Global_var'] = truth_global[N_args:2*N_args].astype(bool)

    # for i in range(M):
    #     R2 = df._append(new_row, ignore_index=True) ## Do more here to add anotehr Row with the RMSE/R2 for each trace
    # df['newlevel'] = 'C'
    # df = df.set_index('newlevel', append=True).unstack('newlevel')
    
    # print(df)
    
    return fit_object, df

## Useful Functions for the Analysis of the 2.7um work

In [ ]:
#| export
def file_loader(input_list:list, # a list of file names to load
                name_list:list, # a list of what we want the new files to be named
                path:str, # a string containing the path to the folder containing the files
                skip_val=1, # by default skip first row of the file
                data_type='pump-probe', # type of data being loaded
               ) -> dict: # returns a dictionary where each key is the name given in name_list and the value is a dataframe with the data
    data = {name:{} for name in name_list}
    
    #bar = alive_it(enumerate(input_list))  # <<-- bar with wrapped items
    
    print(f'There are a total of {len(input_list)} files in the directory')

    with alive_bar(len(input_list),force_tty=True) as bar:
        for i,file in enumerate(input_list):  
            time.sleep(0.005)
    
            #print(f'File: {file} loaded')
        
            df = pd.read_excel(path+file,skiprows=skip_val,header=None)
        
            df2 = data_reformatter(df)

            if data_type=='pump-probe':
                t0_loc = (df2['median'] - np.mean(df2['median'][0:10])).abs().argmax()
            
                t0 = df2['delay'][t0_loc]

            else:
                t0_loc = np.argmax(np.abs(df2['mean']))
                t0 = df2['delay'][t0_loc]
            
            #t0 = df2['delay'][(np.abs(df2['median'][0:len2])).idxmax()]

            data[name_list[i]]['t0'] = t0
            data[name_list[i]]['df'] = df2
            
            bar()

    return data

In [ ]:
#| export

def basic_plot(data_dict,name_array,**kwargs):

    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    if 'm' in list(kwargs.keys()):
        m = kwargs['m']
    else:
        m = 1

    if 'lines' in list(kwargs.keys()):
        if kwargs['lines']=='lines':
            plt.rc('lines', linewidth=1, marker='')
        elif kwargs['lines']=='markers':
            plt.rc('lines', linewidth=0, marker='o',markerfacecolor='white')
        else:
            plt.rc('lines', linewidth=1, marker='o',markerfacecolor='white')
            # plt.rc('markers', marker='o',markerfacecolor='white')
    else:
        marker='o'
        
    fig,ax = plt.subplots()
    
    for name in name_array:
        df = treatment(data_dict[name])
        (xm,ym) = rebin(df['delay'],df['median'],m)
        df_new = pd.DataFrame({'delay':xm,'mean':ym})
        delay = df_new['delay']
        signal = df_new['mean'].to_numpy()
        offset = np.mean(signal[0:int(20/m)])
        signal -= offset
        norm_signal = signal/np.max(np.abs(signal))
        xoffset = delay[np.argmax(np.abs(np.gradient(savgol_filter(norm_signal,20,5))))]
        ### Assume normal situation is to have sign negative on these traces
        sign = int(np.sign(signal[np.argmax(np.abs(signal))]))
        if sign==1:
            new_sign=-1
        else:
            new_sign=1
        ax.plot(delay-xoffset,new_sign*norm_signal,label=name)
        if 'errs' in list(kwargs.keys()):
            ax.plot(delay-xoffset,new_sign*norm_signal,label=name)
        print(f'The time resolution is: {round(1000*(delay[1]-delay[0]),3)} fs') 
    
        # plt.yscale('semilog')
        plt.legend(frameon=False)
        ax.set_xlabel(r'$\mathrm{t_{delay}}$ [ps]')
        ax.set_ylabel(r'$\mathrm{\Delta \sigma}$($\mathrm{t_{delay}}$)')
        if 'xlims' in list(kwargs.keys()):
            (xmin,xmax) = kwargs['xlims']
            ax.set_xlim(xmin,xmax)
        if 'ylims' in list(kwargs.keys()):
            (ymin,ymax) = kwargs['xlims']
            ax.set_ylim(ymin,ymax)

In [ ]:
#| export
def prepulse_analyser(dict_input):

    df_input = dict_input['df']

    t0 = dict_input['t0']

    peak_pos = np.argmax(np.abs(dict_input['df']['mean']-dict_input['df']['mean'][0]))
    
    t_start_point = 0 #np.where(np.abs(df_input['mean']-trange[0])<0.01)[0][0]
    t_end_point = peak_pos# np.where(np.abs(df_input['mean']-trange[1])<0.01)[0][0]

    # subrange = np.arange(t_start_point,t_end_point,1)

    

    scans = [col for col in dict_input['df'] if col.startswith('scan')]

    tdata = {'delay':df_input['delay'] - t0}
    
    df_new = pd.DataFrame(tdata)
    
    for scan in scans:

        y_offset = np.mean(df_input[scan])#np.mean(df_input[scan][int(t_start_point):int(t_end_point)])

        df_new[scan] = df_input[scan] - y_offset

    scan_select = scans#['scan'+str(i) for i in [1,2,3,4,5,6,7,8,9]]
    df_new['mean'] = df_new[scan_select].mean(axis=1)
    df_new['median'] = df_new[scan_select].median(axis=1)
    df_new['std'] = df_new[scan_select].std(axis=1)
    
    return df_new

In [ ]:
#| export
def linear_spacing(df_input,**kwargs):
    
    df_output = df_input.copy()

    if 'col' in list(kwargs.items()):
        scans = [col for col in df_output if col.startswith(kwargs['col'])]
    else:
        scans = [col for col in df_output if col.startswith('median')]

    delay = df_output['delay'].to_numpy()
    delay_shift = np.roll(delay,1)
    t_min = np.min(delay[1:] - delay_shift[1:])
    t_max = np.max(delay[1:] - delay_shift[1:])
    
    new_delay = np.arange(delay[0],delay[len(delay)-1],t_min)
    
    dict_new = {}
    dict_new['delay'] = new_delay

    for scan in scans:
        dict_new[scan] = np.interp(new_delay,delay,df_output[scan])

    df_out = pd.DataFrame(dict_new)

    print(t_min,t_max)

    return df_out 

In [ ]:
#| export
def treatment(dict_input):
    df_input = prepulse_analyser(dict_input) # removes the offset
    df_output = mean_spike(df_input) # calculates the mean, assuming there are some spikes in the data
    return df_output

In [ ]:
#| export
def mean_spike(df_input):

    df_output = df_input.copy()

    scans = [col for col in df_output if col.startswith("scan")]

    mean_val = df_output['mean']

    std_val = df_output['std']

    N = len(mean_val)

    # def spike(x,mval,sval):
    #     for i in range(len(x)):
    #         if (np.abs(x[i]-mval[i])<(3*sval[i])):
    #             val_total = x
    
    
    spike_removed_mean = np.zeros(N)
    spike_removed_std = np.zeros(N)

    for i in range(N):
        dummy = []
        for j,scan in enumerate(scans):
            if ((np.abs(df_output[scan][i]-mean_val[i]))<(3*std_val[i])):
                dummy.append(df_output[scan][i])
            else:
                pass

        spike_removed_mean[i] = np.mean(dummy)
        spike_removed_std[i] = np.std(dummy)
    df_output['mean_spike'] = spike_removed_mean
    df_output['std_spike'] = spike_removed_std

    return df_output


In [ ]:
#| export
def plotting_global(named_function,fit_tuple,name_array,data,**kwargs):

    (fit_obj,df) = fit_tuple
    
    font = {'family' : 'sans-serif',
        'weight' : 'regular',
        'size'   : 12}

    mpl.rc('font', **font)

    colors = plt.cm.jet(np.linspace(0,1,len(name_array)))

    fig = plt.figure(figsize=(6,6))
    gs = fig.add_gridspec(2, hspace=0, height_ratios=[1, 4])
    axs = gs.subplots(sharex=True)

    axs[0].set_ylim([-1e-2,1e-2])
    if 'xlims' in list(kwargs.keys()):
        xmin,xmax = kwargs['xlims']
        axs[1].set_xlim([xmin,xmax])

    if 'ylims' in list(kwargs.keys()):
        ymin,ymax = kwargs['ylims']
        axs[1].set_ylim([ymin,ymax])

    if 'names' in list(kwargs.keys()):
        new_names = kwargs['names']
    else:
        new_names = name_array

    axs[0].set_xlabel(r'delay [ps]')
    axs[1].set_ylabel(r'$\Delta T / T$')

    df_names = {name:{} for name in name_array}
    
    for name in name_array:
        df_names[name] = treatment(data[name])

    for i,name in enumerate(name_array):
        offset = df_names[name]['median'][0:30].mean(axis=0)
        mean_sig = df_names[name]['median']-offset
        std_sig = df_names[name]['std_spike']
        delay1 = df_names[name]['delay']

        if np.abs(np.min(mean_sig))>np.abs(np.max(mean_sig)):
            signage = -1
            fit_obj.popt[i,0]=fit_obj.popt[i,0]
            # fit_obj.popt[i,-1]=0
        else:
            signage = 1
        
        dif = mean_sig-signage*named_function(delay1,*fit_obj.popt[i,:])
        
        axs[1].plot(delay1,mean_sig,color = colors[i],marker='o',linewidth=0,markerfacecolor='white',label=new_names[i])
        # axs[1].fill_between(delay,
        #                     signage*mean_sig+std_sig,
        #                     signage*mean_sig-std_sig,color=colors[i],alpha=0.2)

        # print(fit_obj.popt[i,:])
        # print(name,fit_obj.popt[0])
        axs[1].plot(delay1,signage*named_function(delay1,*fit_obj.popt[i,:]),color = colors[i],linewidth=3)

        axs[0].plot(delay1,dif,color = colors[i],marker='o',linewidth=0,markerfacecolor='white')

        axs[1].set_xlabel('delay [ps]')
        # axs[0].fill_between(delay,
        #                     dif+std_sig,
        #                     dif-std_sig,color=colors[i],alpha=0.2)

    plt.legend(frameon=False)

    if 'save' in list(kwargs.keys()):
        filename_plot = './Presentation/images/'+kwargs['save']+'.svg'
        filename_table = './Presentation/tables/'+kwargs['save']+'.pkl'
        os.makedirs(os.path.dirname(filename_plot), exist_ok=True)
        os.makedirs(os.path.dirname(filename_table), exist_ok=True)
        fig.savefig(filename_plot,bbox_inches='tight')
        df.to_pickle(filename_table)
    return 1
    

In [ ]:
#| export
def global_plot(name_dict,data,guess,**kwargs):

    df_names = {name:{} for name in name_dict}
    
    for name in name_dict:
        df_names[name] = treatment(data[name])

    new_x = {}
    new_y= {}
    
    for i,name in enumerate(name_dict):
        offset = df_names[name]['median'][0:30].mean(axis=0) # y offset

        ### experimental
        df_linear =  linear_spacing(df_names[name],col='median')
        mean_sig = df_linear['median']
        delay = df_linear['delay']
        
        # mean_sig = df_names[name]['median']
        # delay = df_names[name]['delay'].copy()

        if np.abs(np.min(mean_sig-offset))>np.abs(np.max(mean_sig-offset)):
            signage = -1
        else:
            signage = 1

        yval = mean_sig-offset
        
        if 'trange' in list(kwargs.keys()):
            trange=kwargs['trange']
            if type(trange)==tuple:
                tranger = trange
            elif type(trange)==dict:
                tranger = trange[name]
                
            tol = np.abs(delay[1]-delay[0])
            t_start = np.where(np.abs(delay-tranger[0])<tol)[0][0]
            t_end = np.where(np.abs(delay-tranger[1])<tol)[0][0]
            t_start = 0
            delay = delay[t_start:t_end]
            yval = yval[t_start:t_end]
                

        if 'xlims' in list(kwargs.keys()):
            pass
        if 'ylims' in list(kwargs.keys()):
            pass
        if 'save' in list(kwargs.keys()):
            pass
        
        new_x[name] = delay
        new_y[name] = signage*yval

    fit_tuple = global_fit(convolved_biexp_decay_beta, new_x, new_y, guess)

    (fig,df) = fit_tuple

    col_names = [s for s in df.columns if s.startswith('trace')]

    new_col_dict = dict(zip(col_names,name_dict))
    df.rename(columns=new_col_dict,inplace=True)

    tuple_new = (fig,df)

    plotting_global(convolved_biexp_decay_beta,fit_tuple,name_dict,data,**kwargs)
    
    return tuple_new

In [ ]:
#| export
def auto_guess(dict_1,nos_traces):

    ([*keys],vals) = zip(*dict_1.items())
    new_dict = {}
    for i in range(nos_traces):
        for j in range(len(keys)):
            new_keys = keys[j]+str(i)
            new_dict[new_keys] = vals[j]

    return new_dict

In [ ]:
#| export
def file_save(save_name,fig,**kwargs):
    dpi=None
    if 'ext' in list(kwargs.keys()):
        filename_plot = './Presentation/images/'+save_name+'.'+kwargs['ext']
        if kwargs['ext']=='png':
            dpi = 600
    else:
        filename_plot = './Presentation/images/'+save_name+'.svg'

    os.makedirs(os.path.dirname(filename_plot), exist_ok=True)
    fig.savefig(filename_plot, bbox_inches='tight', dpi=dpi, format=kwargs['ext'])
    
    if 'df' in list(kwargs.keys()):
        filename_table = './Presentation/tables/'+save_name+'.pkl'
        os.makedirs(os.path.dirname(filename_table), exist_ok=True)
        kwargs['df'].to_pickle(filename_table)

    return 1

In [ ]:
#| export
def resid_only(named_function,fit_tuple,name_array_sub,full_name_array,data,**kwargs):

    ''' Common to all code '''
    font = {'family' : 'sans-serif',
        'weight' : 'regular',
        'size'   : 12}

    mpl.rc('font', **font)

    colors = ['red','black','green','blue','purple','grey','lightgreen']#plt.cm.jet(np.linspace(0,1,len(name_array_sub)))
    
    df_names = {name:{} for name in name_array_sub}

    print(name_array_sub)
    for name in name_array_sub:
        df_names[name] = treatment(data[name])

    index_pos = [full_name_array.index(x) for x in name_array_sub if x in full_name_array]

    fig,ax = plt.subplots()
    
    if 'xlims' in list(kwargs.keys()):
        xlims = kwargs['xlims']
        ax.set_xlim(*xlims)
    else:
        ax.set_xlim([0,20])
        
    if 'ylims' in list(kwargs.keys()):
        ylims = kwargs['ylims']
        ax.set_ylim(*ylims)

    if 'names' in list(kwargs.keys()):
        names = kwargs['names']
    else:
        names = name_array_sub

    dict_out = {}
    R2_arr = []
    (df,fit_obj) = fit_tuple
    for i,name in enumerate(name_array_sub):
        offset = df_names[name]['median'][0:30].mean(axis=0)
        mean_sig = df_names[name]['median']-offset
        std_sig = df_names[name]['std_spike']
        delay1 = df_names[name]['delay']

        resid = mean_sig - named_function(delay1,*fit_obj.popt[index_pos[i]])
        # ax.plot(delay1,resid,linewidth=0.5,color=colors[i],label=name)
        ax.set_xlabel('time [ps]')
        ax.set_ylabel('residual')

        if 'histogram' in list(kwargs.keys()):
            df_hist = pd.DataFrame({'val':mean_sig.to_numpy(),'delay':delay1}) 
            sns.displot(df_hist, x="val", binwidth=0.0005)
        
        if 'fit' in list(kwargs.keys()):
            named_function2 = kwargs['fit']
            if 'guess' in list(kwargs.keys()):
                guess = kwargs['guess']
            else:
                if named_function2==cos_fn:            
                    guess = [0.16,np.max(resid),0,.001]
                if named_function2==cos_fn2:
                    guess = [0.08,0.8*np.max(resid),np.max(resid)*0.2,0.1,.001]
                if named_function2==cos_fn3:
                    guess = [0.08,0.6*np.max(resid),0.3*np.max(resid),0.1*np.max(resid),0.1,.001]
            tol=np.abs(delay1[1]-delay1[0])
            t_start_pt = np.where(np.abs(delay1-xlims[0])<tol)[0][0]
            t_end_pt = np.where(np.abs(delay1-xlims[1])<tol)[0][0]
            popt,pcov = curve_fit(named_function2,delay1[t_start_pt:t_end_pt],resid[t_start_pt:t_end_pt],p0=guess,method='lm')
            offset = popt[len(popt)-1]
            
            ax.plot(delay1,resid-offset,linewidth=0.1,alpha=1,color=colors[i])
            ax.plot(delay1,resid-offset,marker='o',linewidth=0,markerfacecolor='white',color=colors[i],label=names[i])
            
            
            ax.plot(delay1,named_function2(delay1,*popt)-offset,color=colors[i],linewidth=2)
            fit_obj2 = fit_result(delay1[t_start_pt:t_end_pt],resid[t_start_pt:t_end_pt],named_function2,popt,pcov)
            # print(popt)
            variables = inspect.getfullargspec(named_function2).args
            variables = variables[1:]
            #outputs = [names[i]]
            dict_out[names[i]] = zip(np.round(fit_obj2.popt,3),np.round(fit_obj2.pstd,3))
            R2_arr.append(fit_obj2.R2)
            
            print(f'R2 = {round(fit_obj2.R2_adj,3)}')
        else:
            ax.plot(delay1,resid,linewidth=0.1,alpha=1,color=colors[i])
            ax.plot(delay1,resid,marker='o',linewidth=0,markerfacecolor='white',color=colors[i],label=names[i])
        plt.legend(frameon=False,loc='upper left')

    print(dict_out)
    df_out = pd.DataFrame(dict_out,index=variables)
    df_out.loc['R2'] = R2_arr
        
    if 'save' in list(kwargs.keys()):
        save_name = kwargs['save']
        file_save(save_name,fig,df=df_out)

    print(df_out)
    return df_out
    

In [ ]:
#| export
def cos_fn(x,f,A,x0,y0):
    # A,f,phi,y0 = args
    return A*np.cos(2*np.pi*f*(x-x0))+y0

## Functions for use with un-excited THz-TDS data

In [ ]:
#| export
def electric_field(input_data,
                   conversion_scale, #from oscilloscope magnitude to LIA reading
                   lambda0, # in nm
                   Si_in=True):

    n0 = 2.7830
    n_air = 1.0
    r41 = 4.04e-12 # m/V 
    L = 400e-6 # m
    lambda0 *= 1e-9 # m
    t_refl = (n0 - n_air)/(n0 + n_air)

    E_out_scale = lambda0 / (2*np.pi*n0**3*r41*t_refl*L)

    E_out = np.arcsin(input_data*1e-3*conversion_scale/12) * E_out_scale

    scale=1
    if Si_in==True:
        scale = 0.8

    return (E_out/scale)/1e5 # kV/cm

In [ ]:
#| export
def data_reformatter(df,**kwargs):

    """
    The purpose of this function is to repackage or change the format of the data, 
    such that rather than having two columns with a large number of rows, we have
    different columns for each scan. The data is also augmented with two additional 
    rows - the mean and the std of the scans.

    kwargs here are used to either correct result to E(kV) or not
    """
    if kwargs['convert_to_E']==True:
        scale_LIA = kwargs['conversion_scale']
        lambda0 = kwargs['lambda0']
        Si_in = kwargs['Si_in']
    else:
        scale_LIA = 1.0
        
    data = df.to_numpy()

    Ntot,_ = np.shape(data) # total number of rows

    delays = np.unique(data[:,0]) # unique delay positions in mm
    
    delays = 2*delays/(consts.c) * 1e-3 * 1e12
    
    N = len(delays) # number of delays per scan

    M = int(Ntot/N) # number of scans

    df2 = pd.DataFrame(np.reshape(data[:,1],(M,N)).T) # reshape the data and put it into a dataframe

    col_names = {i:f'scan{i}' for i in range(M)} # create an array with names like 'scan0, scan1, ....'
    
    df2 = df2.rename(columns=col_names) # rename the columns

    for col in col_names:
        df2[col] = electric_field(df2[col],
                                  scale_LIA,
                                  lambda0,
                                  Si_in)

    mean_col = df2.mean(axis=1) # calculate a new column containing the mean value for each delay

    median_col = df2.median(axis=1)

    std_col = df2.std(axis=1) # calculate a new column containing the std value for each delay

    df2['mean'] = mean_col # augment the dataframe with a new column containing the mean

    df2['median'] = median_col
    
    df2['std'] = std_col # augment the dataframe with a new column containing the std

    df2['se'] = std_col / np.sqrt(M)
    
    df2.insert(loc=0,column='delay',value=delays) # add a new column at the start of the DataFrame containing the delays

    return df2

In [ ]:
#| export
def fft_simple(df,**kwargs):
    """
    This is a simple implementation of fft that will work for a DataFrame object
    """
    # temporal resolution
    dt = np.abs(df['delay'][1] - df['delay'][0])
    
    # sampling rate
    S = 1/dt

    pad = kwargs['pad']

    N_old,M = np.shape(df)

    scan_cols = [s for s in df if s.startswith('scan')]
    
    M = len(scan_cols)
    
    scans = df[scan_cols].to_numpy()

    zero_pad = np.zeros((pad,M))

    data_pad = np.vstack((scans,zero_pad)) # just stacking at end, should probably zero pad both sides

    ##t_pad = np.array([df['delay'][0] + dt*i for i in range(N_old+pad)])

    freqs = fftfreq(N_old+pad,dt)
    
    # FFT of signal
    N_freq = len(freqs)
    
    N2 = int(np.floor(N_freq/2))

    signal = fftn(data_pad,axes=0)

    PSD = np.zeros(np.shape(signal))

    arg_cmplx = np.zeros(np.shape(signal))
    
    for i in range(np.size(signal,1)):
        PSD[:,i] = np.abs(signal[:,i])**2
        arg_cmplx[:,i] = np.unwrap(np.angle(signal[:,i]))

    df2 = pd.DataFrame(signal[1:N2])

    col_names = {i:f'scan{i}' for i in range(M)}

    cols = ['scan'+str(i) for i in range(M)]
    
    df2.rename(columns=col_names,inplace=True)

    mean_col = df2.mean(axis=1)

    std_col = np.std(df2.to_numpy(),axis=1)

    df2.insert(loc=0,column='freq',value=freqs[1:N2])

    df2['mean'] = mean_col

    df2['std'] = std_col

    df2['PSD'] = np.mean(PSD[1:N2],axis=1)

    df2['PSD_err'] = np.std(PSD[1:N2],axis=1)

    df2['arg'] = np.mean(arg_cmplx[1:N2],axis=1)

    df2['arg_err'] = np.std(arg_cmplx[1:N2],axis=1)
    
    return df2

In [ ]:
#| export
def transmission(df1,df2):

    cols = [s for s in df1 if s.startswith('scan')]
    
    M = len(cols)

    col_names = {i:'scan'+str(i) for i in range(M)}
    
    df_T = df1[cols].to_numpy()

    ref_mean = df2['mean'].to_numpy()

    dnew = np.zeros((np.size(df_T,0),M),dtype=np.complex128)
    
    for i in range(M):

        temp = df_T[:,i]/ref_mean
        
        tmod = np.abs(df_T[:,i]/ref_mean)
                
        dnew[:,i] = temp

    T_mean = np.mean(dnew,axis=1)
    
    T_std = np.std(dnew,axis=1)
    
    df = pd.DataFrame(dnew)

    df.rename(columns=col_names,inplace=True)

    df['T'] = T_mean

    df['T_err'] = T_std
    
    df.insert(loc=0,column='freq',value=df1['freq'])

    return df

In [ ]:
#| export
def conductivity(T_in,**kwargs):
    """
    This function calculates the complex conductivity according to equation 1 or 2
    """
    n_substrate = 3.45+0.25j # Assuming a constant, real refractive index for the substrate. This is not true, but need to dig up the measured data.
    Z_0 = 377 # ohms, vacuum permittivity
    n_A = n_substrate + 1 # from equation section
    n_B = n_substrate + 1 # from equation section
    e2_h = consts.e**2/consts.h # factor that converts to units of e^2/h
    
    cols = [s for s in T_in if s.startswith('scan')]
    
    M = len(cols)

    col_names = {i:'scan'+str(i) for i in range(M)}
    
    data_in = T_in[cols].to_numpy()

    dnew = np.zeros((np.size(data_in,0),M),dtype=np.complex128)

    if (kwargs["equation"]=="eq1"):
        
        for i in range(M):
            
            dnew[:,i] = (1/e2_h) * n_A/Z_0 * ((1/data_in[:,i]) - 1)

    if (kwargs["equation"]=="eq2"):

        for i in range(M):
            
            dnew[:,i] = (1/e2_h) * (n_A * np.sqrt(n_A**2 + 4*n_B*(n_A+n_B)*data_in[:,i]) - \
            n_A**2 - 2*n_A*n_B*data_in[:,i])/(2*n_B*Z_0*data_in[:,i])
            
    sigma_mean = np.mean(dnew,axis=1)
    
    sigma_std = np.std(dnew,axis=1)
    
    df = pd.DataFrame(dnew)

    df.rename(columns=col_names,inplace=True)

    df['mean'] = sigma_mean

    df['std'] = sigma_std

    df['real'] = np.mean(np.real(dnew),axis=1)

    df['real_err'] = np.std(np.real(dnew),axis=1)

    df['imag'] = np.mean(np.imag(dnew),axis=1)

    df['imag_err'] = np.std(np.imag(dnew),axis=1)

    return df

In [ ]:
#| export
def four_point(x:np.ndarray, # Input array that contains the independent variable.
    mu_e:np.double, # electron mobility
    mu_h:np.double, # hole mobility
    dn:np.double, # non-zero conductivity at CNP
    rho_s:np.double, # Gate indpendent resistivity given by short range scattering in graphene
    n_0:np.double, # carrier density at Vg=0 (intrinsic doping)
    **kwargs) ->np.ndarray: # A new float array containing the conductivity.
    #J. H. Gosling, O. Makarovsky, F. Wang, et al., “Universal mobility characteristics of graphene originating from charge scattering by ionised impurities,” Commun. Phys. 4 (2021).593

    ## Constants used ##
    e = 1.6e-19 # electron charge
    eps_0 = 8.8541878188e-12 # F/m, vacuum permittivity
    A = 7.0 * 7.0 # mm2, device area
    d = 300 # nm, SiO2 thickness
    eps_sio2 = 3.9 # dielectric constant of SiO2
    C = eps_sio2 * eps_0 * A * 1e-6 / (d*1e-9) # capacitance in Farads, F
    
    n_C = C * x / e # geometric carrier concentration using parralel plate capacitance model

    V_CNP = kwargs['CNP'] # not sure if this bit of code is going to work

    if (x < V_CNP):
        
        mu = mu_h
        
    elif (x > V_CNP):
        
        mu = mu_e

    n_carrier = np.abs(((1/n_C) + (e*mu/rho_s))**(-1) + n_0)

    
    if (n_carrier < (dn/2)):

        sigma = e * mu * (dn/4 + (n_carrier**2)/dn)

    elif (n_carrier > (dn/2)):

        sigma = e * mu * (dn/4 + (n_carrier**2)/dn)        


    return sigma


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def foo(): pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
def file_save(save_name,fig,**kwargs):
    if 'file_type' in list(kwargs.keys()):
        filename_plot = './Presentation/images/'+save_name+kwargs['file_type']
    else:
        filename_plot = './Presentation/images/'+save_name+'.svg'
    
    if 'df' in list(kwargs.keys()):
        filename_table = './Presentation/tables/'+save_name+'.pkl'
        os.makedirs(os.path.dirname(filename_table), exist_ok=True)
        kwargs['df'].to_pickle(filename_table)

    os.makedirs(os.path.dirname(filename_plot), exist_ok=True)
    
    fig.savefig(filename_plot,bbox_inches='tight')
    
    print(f'Figure saved successfully at {filename_plot}')